In [1]:
import pandas as pd

In [5]:
resident = pd.read_csv('C:/Users/bhg/Desktop/2022-BigContest/2022빅콘/2022빅콘테스트_데이터분석리그_데이터분석분야_챔피언부문_데이터셋_220908/1.ev_app_resident.csv')
activity = pd.read_csv('C:/Users/bhg/Desktop/2022-BigContest/2022빅콘/2022빅콘테스트_데이터분석리그_데이터분석분야_챔피언부문_데이터셋_220908/2.ev_app_activity.csv')
activity_resident = pd.read_csv('C:/Users/bhg/Desktop/2022-BigContest/2022빅콘/2022빅콘테스트_데이터분석리그_데이터분석분야_챔피언부문_데이터셋_220908/3.ev_app_activity_resident.csv')

In [ ]:
resident.drop(['ccw_cd','adng_cd'],axis=1,inplace=True)
activity.drop(['ccw_cd','adng_cd'],axis=1,inplace=True)
activity_resident.drop(['ccw_cd','adng_cd','mega_cd','mega_nm'],axis=1,inplace=True)

### 시간대별 집계

In [4]:
r_timezone = resident.groupby(['cell_id','time_zone'])['count_cust'].sum().reset_index()
r_timezone

,cell_id,time_zone,count_cust
0,82743412,3,4.882
1,82807429,1,90.944
2,82807429,2,68.208
3,82807429,3,56.840
4,82807429,4,142.100
...,...,...,...
38832,93750253,3,5.521
38833,93750253,5,5.521
38834,93830213,3,5.022
38835,93830213,5,5.022


In [5]:
r_timezone = pd.merge(r_timezone,resident,how='left',on=['cell_id','time_zone'])
r_timezone.drop(['base_dt','count_cust_y','gender','age','app_web','dow'],axis=1,inplace=True) 
r_timezone.drop_duplicates(keep='first',inplace=True) # time_zone

In [6]:
r_timezone

,cell_id,time_zone,count_cust_x,ccw_nm,adng_nm,cell_xcrd,cell_ycrd
0,82743412,3,4.882,용인시,동천동,127.03221,37.360703
1,82807429,1,90.944,용인시,동천동,127.03441,37.370174
13,82807429,2,68.208,용인시,동천동,127.03441,37.370174
23,82807429,3,56.840,용인시,동천동,127.03441,37.370174
32,82807429,4,142.100,용인시,동천동,127.03441,37.370174
...,...,...,...,...,...,...,...
292038,93750253,3,5.521,용인시,백암면,127.42089,37.149320
292039,93750253,5,5.521,용인시,백암면,127.42089,37.149320
292040,93830213,3,5.022,용인시,백암면,127.42372,37.133545
292041,93830213,5,5.022,용인시,백암면,127.42372,37.133545


In [7]:
r_timezone.to_csv('r_timezone.csv',index=False)

In [8]:
a_timezone = activity.groupby(['cell_id','time_zone'])['count_cust'].sum()
a_timezone = pd.merge(a_timezone,activity,how='left',on=['cell_id','time_zone'])
a_timezone.drop(['base_dt','count_cust_y','gender','age','app_web','dow'],axis=1,inplace=True) 
a_timezone.drop_duplicates(keep='first',inplace=True) # time_zone

In [9]:
a_timezone.to_csv('a_timezone.csv',index=False)

### 격자에 따라 세 데이터 merge

In [6]:
merged1 = pd.merge(resident,activity,how='outer',on=['cell_id','ccw_nm','adng_nm','cell_xcrd','cell_ycrd','base_dt','time_zone','dow']) 
# 같은 격자내 같은 날짜, 시간대는 묶기

In [7]:
merged2 = pd.merge(merged1,activity_resident, how='outer',on=['cell_id','ccw_nm','adng_nm','cell_xcrd','cell_ycrd','base_dt','time_zone','dow'])
merged2_y = merged2.loc[merged2['ccw_nm']=='용인시'].copy()
merged2_y

,base_dt,dow,ccw_cd_x,ccw_nm,adng_cd_x,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender_x,...,app_web_y,count_cust_y,mega_cd,mega_nm,ccw_cd,adng_cd,gender,age,app_web,count_cust
0,20220605,7,4146.0,용인시,41463572.0,동백1동,86135034,127.15225,37.286300,MALE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20220625,6,4146.0,용인시,41465580.0,상현2동,84375196,127.09008,37.309536,MALE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20220625,6,4146.0,용인시,41465580.0,상현2동,84375196,127.09008,37.309536,MALE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20220614,2,4146.0,용인시,41461250.0,포곡읍,87958893,127.21660,37.274303,FEMALE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20220619,7,4146.0,용인시,41465530.0,신봉동,84119243,127.08097,37.323480,MALE,...,EV라운지,4.335,41.0,경기도,4146.0,41465530.0,MALE,13.0,EV라운지,4.335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630223,20220620,1,NaN,용인시,NaN,양지면,90182654,127.29511,37.229393,NaN,...,EV라운지,3.274,41.0,경기도,4146.0,41461360.0,FEMALE,4.0,EV라운지,3.274
630224,20220608,3,NaN,용인시,NaN,남사읍,85878686,127.14400,37.122220,NaN,...,EV라운지,4.140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
630225,20220628,2,NaN,용인시,NaN,성복동,83687264,127.06575,37.320717,NaN,...,EV라운지,5.266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
630226,20220628,2,NaN,용인시,NaN,신봉동,84103247,127.08040,37.324825,NaN,...,EV라운지,21.564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
merged2_y.to_csv('bigcon_merged.csv',encoding='cp949')

In [9]:
merged_grids = merged2_y[['adng_nm','cell_id','cell_xcrd','cell_ycrd']].drop_duplicates()

In [10]:
merged_grids.to_csv('merged_grids.csv',encoding='cp949')

#### 격자별 count_cust 합 

In [11]:
grid_countcust = merged2_y.groupby(['adng_nm','cell_id','cell_xcrd','cell_ycrd']).sum()

In [12]:
grid_countcust.reset_index(inplace=True)
grid_countcust['tot_count_cust'] = grid_countcust['count_cust_x'] + grid_countcust['count_cust_y']+ grid_countcust['count_cust']
grid_countcust = grid_countcust[['adng_nm','cell_id','cell_xcrd','cell_ycrd','tot_count_cust']]

In [13]:
grid_countcust

,adng_nm,cell_id,cell_xcrd,cell_ycrd,tot_count_cust
0,구갈동,84919076,127.109460,37.270844,923.508
1,구갈동,84935076,127.110020,37.271297,3.048
2,구갈동,84935081,127.110010,37.273550,1262.039
3,구갈동,84935082,127.110010,37.274000,275.330
4,구갈동,84935083,127.110010,37.274452,181.521
...,...,...,...,...,...
18542,풍덕천2동,84647202,127.099620,37.319935,157.781
18543,풍덕천2동,84663200,127.100180,37.319490,2775.626
18544,풍덕천2동,84679200,127.100746,37.319940,8.130
18545,풍덕천2동,84695200,127.101300,37.320393,10.038


#### 격자, 행정동 인구 결합

In [227]:
pop1 = pd.read_csv("C:/Users/bhg/Desktop/2022-BigContest/2022빅콘/조사 데이터/(용인시)인구 및 세대현황(8월 말 기준).csv",encoding='cp949')
pop2 = pd.read_csv("C:/Users/bhg/Desktop/2022-BigContest/2022빅콘/조사 데이터/(용인시)연령별 인구 현황(8월 말 기준) (1).csv",encoding='cp949')
pop2['읍면동'] = pop2['읍면동'].str.replace(' ','')
pop = pd.merge(pop1,pop2,on='읍면동')
pop.rename(columns={'읍면동':'읍면동_인구',' 합     계 ':'합계_인구'},inplace=True)
# 인구 합계에 외국인은 빠져있음
pop

,읍면동_인구,내국인 계,내국인 남,내국인 여,외국인 계,외국인 남,외국인 여,합계_인구,10대 미만,10대,20대,30대,40대,50대,60대,70대,80대,90대,90세,100세 이상
0,포곡읍,"33,969","17,581","16,388",568,332,236,"33,969","1,992","3,844","4,926","3,764","5,865","6,173","4,173","2,053","1,034",144,28,1
1,모현읍,"26,971","14,123","12,848","1,286",879,407,"26,971","1,762","2,410","3,370","2,944","4,542","4,869","4,047","1,974",908,142,32,3
2,이동읍,"20,555","10,546","10,009","1,134",839,295,"20,555","1,359","1,768","2,149","2,358","3,253","3,805","3,307","1,567",845,137,35,7
3,남사읍,"23,761","12,008","11,753",776,571,205,"23,761","2,953","2,281","1,669","3,783","4,537","3,064","3,046","1,663",665,98,24,2
4,원삼면,"8,133","4,224","3,909",519,406,113,"8,133",321,546,692,653,"1,000","1,551","1,819",956,513,79,17,3
5,백암면,"9,113","4,999","4,114",634,525,109,"9,113",222,"1,115",889,727,969,"1,516","1,932",970,685,86,22,2
6,양지면,"20,989","11,408","9,581",517,366,151,"20,989","1,151","3,020","2,580","2,481","3,082","3,541","3,059","1,261",686,122,30,6
7,중앙동,"25,123","12,758","12,365","1,784","1,006",778,"25,123","1,467","2,102","4,347","3,270","3,794","4,298","3,351","1,608",760,124,39,2
8,역북동,"31,935","16,126","15,809",625,326,299,"31,935","3,342","3,207","5,303","5,353","5,719","4,283","2,775","1,335",543,72,13,3
9,삼가동,"12,508","6,146","6,362",93,40,53,"12,508",952,"1,252","1,810","1,574","2,030","2,317","1,477",695,350,51,10,0


In [228]:
grids = pd.merge(grid_countcust,pop,left_on='adng_nm',right_on='읍면동_인구')
grids.drop('adng_nm',axis=1,inplace=True)

#### 격자, 차량 결합

In [229]:
cars = pd.read_csv("C:/Users/bhg/Desktop/2022-BigContest/2022빅콘/조사 데이터/자동차등록현황(2022년 9월).csv",encoding='cp949')
cars.rename(columns={'구  분':'동리명_차량','합  계':'합계_차량'},inplace=True)
cars

,동리명_차량,합계_차량,승 용,승 합,화 물,특 수,관 용,자가용,영업용
0,포곡읍,18048,14071,739,3110,128,10,17388,650
1,모현읍,16486,12192,720,3507,67,0,15616,870
2,이동읍,13126,9567,385,3129,45,11,12158,957
3,남사읍,13772,11156,352,2208,56,28,13490,254
4,원삼면,5862,3973,146,1704,39,2,5354,506
5,백암면,6491,3607,255,2467,162,1,5225,1265
6,양지면,12941,9713,409,2699,120,0,11940,1001
7,김량장동,9422,7930,358,1098,36,24,9094,304
8,남동,3792,2584,763,435,10,0,3062,730
9,역북동,14184,12574,338,1208,64,85,13698,401


#####  법정동, 행정동 매핑

In [216]:
dongs = pd.read_csv("C:/Users/bhg/Desktop/2022-BigContest/2022빅콘/조사 데이터/법정동행정동매핑.csv",encoding='cp949')
dongs.rename(columns={'읍면동명':'읍면동_매핑'},inplace=True)
dongs

,시군구명,읍면동_매핑,동리명,Unnamed: 3
0,용인시 처인구,포곡읍,포곡읍,NaN
1,용인시 처인구,포곡읍,삼계리,NaN
2,용인시 처인구,포곡읍,금어리,NaN
3,용인시 처인구,포곡읍,둔전리,NaN
4,용인시 처인구,포곡읍,영문리,NaN
...,...,...,...,...
117,용인시 수지구,동천동,고기동,NaN
118,용인시 수지구,상현1동,상현동,NaN
119,용인시 수지구,상현2동,상현동,NaN
120,용인시 수지구,상현3동,상현동,NaN


In [249]:
mapping_dongs_cars= pd.merge(dongs, cars,how='outer',left_on='동리명',right_on='동리명_차량')
# mapping_dong[['읍면동_매핑','동리명','동리명_차량']]
mapping_dongs_cars.dropna(subset=['동리명_차량'],inplace=True)
mapping_dongs_cars.drop(['시군구명','동리명','Unnamed: 3'],axis=1,inplace=True)
mapping_dongs_cars

mapping_total = pd.merge(mapping_dongs_cars,pop,left_on='읍면동_매핑',right_on='읍면동_인구')
mapping_total.drop(['읍면동_매핑'],axis=1,inplace=True)
mapping_total
# 격자는 읍면동 기준!
# 결과 살펴보니 인구의 읍면동이 차량의 동리명보다 크다 -> 차량이 합쳐져서 각 동에 들어가야 함
# 그러나 이상하게 2개의 동리명이 3개의 읍면동으로 들어가는 경우 있어
# ex) 중동과 동백동(동리명)이 동백1,2,3동
# 그냥 각 읍면동 내 동리명 차량의 평균을 차량대수로 보는 것이 맞아보임

,동리명_차량,합계_차량,승 용,승 합,화 물,특 수,관 용,자가용,영업용,읍면동_인구,...,20대,30대,40대,50대,60대,70대,80대,90대,90세,100세 이상
0,포곡읍,18048.0,14071.0,739.0,3110.0,128.0,10.0,17388.0,650.0,포곡읍,...,"4,926","3,764","5,865","6,173","4,173","2,053","1,034",144,28,1
1,모현읍,16486.0,12192.0,720.0,3507.0,67.0,0.0,15616.0,870.0,모현읍,...,"3,370","2,944","4,542","4,869","4,047","1,974",908,142,32,3
2,이동읍,13126.0,9567.0,385.0,3129.0,45.0,11.0,12158.0,957.0,이동읍,...,"2,149","2,358","3,253","3,805","3,307","1,567",845,137,35,7
3,남사읍,13772.0,11156.0,352.0,2208.0,56.0,28.0,13490.0,254.0,남사읍,...,"1,669","3,783","4,537","3,064","3,046","1,663",665,98,24,2
4,원삼면,5862.0,3973.0,146.0,1704.0,39.0,2.0,5354.0,506.0,원삼면,...,692,653,"1,000","1,551","1,819",956,513,79,17,3
5,백암면,6491.0,3607.0,255.0,2467.0,162.0,1.0,5225.0,1265.0,백암면,...,889,727,969,"1,516","1,932",970,685,86,22,2
6,양지면,12941.0,9713.0,409.0,2699.0,120.0,0.0,11940.0,1001.0,양지면,...,"2,580","2,481","3,082","3,541","3,059","1,261",686,122,30,6
7,김량장동,9422.0,7930.0,358.0,1098.0,36.0,24.0,9094.0,304.0,중앙동,...,"4,347","3,270","3,794","4,298","3,351","1,608",760,124,39,2
8,남동,3792.0,2584.0,763.0,435.0,10.0,0.0,3062.0,730.0,중앙동,...,"4,347","3,270","3,794","4,298","3,351","1,608",760,124,39,2
9,역북동,14184.0,12574.0,338.0,1208.0,64.0,85.0,13698.0,401.0,역북동,...,"5,303","5,353","5,719","4,283","2,775","1,335",543,72,13,3


In [250]:
grids['읍면동_인구'].unique()

array(['구갈동', '구성동', '기흥동', '남사읍', '동백1동', '동백2동', '동백3동', '동부동', '동천동',
       '마북동', '모현읍', '백암면', '보라동', '보정동', '상갈동', '상하동', '상현1동', '상현2동',
       '서농동', '성복동', '신갈동', '신봉동', '양지면', '영덕1동', '영덕2동', '원삼면', '유림동',
       '이동읍', '죽전1동', '죽전2동', '중앙동', '포곡읍', '풍덕천1동', '풍덕천2동'],
      dtype=object)

In [248]:
mapping_total.to_csv('mapping_total.csv',encoding='cp949')

### resident, activity_resident 차이 확인

In [48]:
a = activity_resident.loc[activity_resident['ccw_nm']=='용인시']
b = resident.copy()

In [15]:
aandb = pd.merge(a,b,how='inner',on=['base_dt','dow','cell_id','cell_xcrd','cell_ycrd','ccw_nm','adng_nm','gender','age','app_web','time_zone']) # 둘 다 포함된 데이터 
aorb = pd.merge(a,b,how='outer',on=['base_dt','dow','cell_id','cell_xcrd','cell_ycrd','ccw_nm','adng_nm','gender','age','app_web','time_zone'],indicator=True) 

In [49]:
# aandb = pd.merge(a,b,how='inner',on=['cell_id','ccw_nm','adng_nm','cell_xcrd','cell_ycrd','base_dt','time_zone','dow'])
# aorb = pd.merge(a,b,how='outer',on=['cell_id','ccw_nm','adng_nm','cell_xcrd','cell_ycrd','base_dt','time_zone','dow'],indicator=True) 

In [50]:
aandb.rename(columns={'count_cust_x':'ra_count_cust','count_cust_y':'r_count_cust'})

,base_dt,dow,mega_nm,ccw_nm,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender_x,age_x,app_web_x,time_zone,ra_count_cust,gender_y,age_y,app_web_y,r_count_cust
0,20220608,3,경기도,용인시,신갈동,84951117,127.110490,37.290226,FEMALE,13,EV라운지,3,6.664,FEMALE,13,EV라운지,6.664
1,20220607,2,경기도,용인시,신갈동,84887106,127.108270,37.283460,FEMALE,8,EV라운지,5,4.042,FEMALE,8,EV라운지,4.042
2,20220614,2,경기도,용인시,죽전2동,84967209,127.110840,37.332140,MALE,8,EV라운지,3,4.178,MALE,8,EV라운지,4.178
3,20220602,4,경기도,용인시,죽전1동,85063199,127.114235,37.330350,MALE,3,EV라운지,6,5.405,MALE,3,EV라운지,5.405
4,20220602,4,경기도,용인시,죽전1동,85063199,127.114235,37.330350,MALE,3,EV라운지,6,5.405,MALE,9,EV라운지,4.320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179169,20220613,1,경기도,용인시,신봉동,83703275,127.066284,37.326130,FEMALE,11,EV라운지,4,10.188,MALE,10,하이차저,3.819
179170,20220613,1,경기도,용인시,신봉동,83703275,127.066284,37.326130,FEMALE,11,EV라운지,4,10.188,FEMALE,11,EV라운지,10.188
179171,20220629,3,경기도,용인시,상갈동,84935061,127.110060,37.264534,MALE,4,EV라운지,1,3.985,MALE,4,EV라운지,3.985
179172,20220610,5,경기도,용인시,상하동,85750998,127.138840,37.259216,FEMALE,13,EV라운지,1,14.516,FEMALE,13,EV라운지,14.516


In [51]:
aorb.rename(columns={'count_cust_x':'ra_count_cust','count_cust_y':'r_count_cust'})

,base_dt,dow,mega_nm,ccw_nm,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender_x,age_x,app_web_x,time_zone,ra_count_cust,gender_y,age_y,app_web_y,r_count_cust,_merge
0,20220611,6,경기도,용인시,모현읍,86695054,127.171880,37.311146,FEMALE,13.0,EV라운지,1,4.492,NaN,NaN,NaN,NaN,left_only
1,20220608,3,경기도,용인시,신갈동,84951117,127.110490,37.290226,FEMALE,13.0,EV라운지,3,6.664,FEMALE,13.0,EV라운지,6.664,both
2,20220608,3,경기도,용인시,보라동,85175019,127.118576,37.252396,MALE,12.0,EV라운지,6,4.129,NaN,NaN,NaN,NaN,left_only
3,20220607,2,경기도,용인시,신갈동,84887106,127.108270,37.283460,FEMALE,8.0,EV라운지,5,4.042,FEMALE,8.0,EV라운지,4.042,both
4,20220614,2,경기도,용인시,죽전2동,84967209,127.110840,37.332140,MALE,8.0,EV라운지,3,4.178,MALE,8.0,EV라운지,4.178,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416709,20220616,4,NaN,용인시,모현읍,89302948,127.263780,37.337055,NaN,NaN,NaN,5,NaN,MALE,11.0,EV라운지,3.247,right_only
416710,20220617,5,NaN,용인시,포곡읍,88566830,127.238070,37.263084,NaN,NaN,NaN,3,NaN,MALE,3.0,EV라운지,4.463,right_only
416711,20220610,5,NaN,용인시,기흥동,84679026,127.101160,37.241520,NaN,NaN,NaN,5,NaN,FEMALE,10.0,EV라운지,3.227,right_only
416712,20220605,7,NaN,용인시,상현1동,84071205,127.079384,37.304993,NaN,NaN,NaN,3,NaN,MALE,2.0,EV라운지,5.082,right_only


In [52]:
aorb

,base_dt,dow,mega_nm,ccw_nm,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender_x,age_x,app_web_x,time_zone,count_cust_x,gender_y,age_y,app_web_y,count_cust_y,_merge
0,20220611,6,경기도,용인시,모현읍,86695054,127.171880,37.311146,FEMALE,13.0,EV라운지,1,4.492,NaN,NaN,NaN,NaN,left_only
1,20220608,3,경기도,용인시,신갈동,84951117,127.110490,37.290226,FEMALE,13.0,EV라운지,3,6.664,FEMALE,13.0,EV라운지,6.664,both
2,20220608,3,경기도,용인시,보라동,85175019,127.118576,37.252396,MALE,12.0,EV라운지,6,4.129,NaN,NaN,NaN,NaN,left_only
3,20220607,2,경기도,용인시,신갈동,84887106,127.108270,37.283460,FEMALE,8.0,EV라운지,5,4.042,FEMALE,8.0,EV라운지,4.042,both
4,20220614,2,경기도,용인시,죽전2동,84967209,127.110840,37.332140,MALE,8.0,EV라운지,3,4.178,MALE,8.0,EV라운지,4.178,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416709,20220616,4,NaN,용인시,모현읍,89302948,127.263780,37.337055,NaN,NaN,NaN,5,NaN,MALE,11.0,EV라운지,3.247,right_only
416710,20220617,5,NaN,용인시,포곡읍,88566830,127.238070,37.263084,NaN,NaN,NaN,3,NaN,MALE,3.0,EV라운지,4.463,right_only
416711,20220610,5,NaN,용인시,기흥동,84679026,127.101160,37.241520,NaN,NaN,NaN,5,NaN,FEMALE,10.0,EV라운지,3.227,right_only
416712,20220605,7,NaN,용인시,상현1동,84071205,127.079384,37.304993,NaN,NaN,NaN,3,NaN,MALE,2.0,EV라운지,5.082,right_only


In [53]:
aorb.loc[aorb['_merge']=='left_only'].to_csv('test.csv',index=False)
# resident_activity에만 찍히는 것은 혹시 외곽으로, 중심점은 용인이지만 격자 자체는 다른 시에도 포함되어 있는 것이 아닐까? 했으나 그렇지 않음

In [54]:
aorb.loc[aorb['_merge']!='both'].sort_values(by='cell_id')

,base_dt,dow,mega_nm,ccw_nm,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender_x,age_x,app_web_x,time_zone,count_cust_x,gender_y,age_y,app_web_y,count_cust_y,_merge
111504,20220629,3,경기도,용인시,동천동,82695419,127.03050,37.362500,MALE,12.0,EV라운지,5,5.684,NaN,NaN,NaN,NaN,left_only
248735,20220601,3,경기도,용인시,동천동,82695419,127.03050,37.362500,MALE,12.0,EV라운지,4,5.684,NaN,NaN,NaN,NaN,left_only
40348,20220601,3,경기도,용인시,동천동,82695419,127.03050,37.362500,MALE,12.0,EV라운지,2,5.684,NaN,NaN,NaN,NaN,left_only
200965,20220628,2,경기도,용인시,동천동,82695419,127.03050,37.362500,MALE,12.0,EV라운지,5,5.684,NaN,NaN,NaN,NaN,left_only
241662,20220614,2,경기도,용인시,동천동,82695419,127.03050,37.362500,MALE,12.0,EV라운지,2,5.684,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94211,20220601,3,경기도,용인시,백암면,93782248,127.42202,37.147970,MALE,8.0,EV Infra,2,4.078,NaN,NaN,NaN,NaN,left_only
132454,20220603,5,경기도,용인시,백암면,93782250,127.42202,37.148870,FEMALE,12.0,EV라운지,3,6.577,NaN,NaN,NaN,NaN,left_only
354771,20220616,4,NaN,용인시,백암면,93830213,127.42372,37.133545,NaN,NaN,NaN,5,NaN,FEMALE,12.0,EV라운지,5.022,right_only
355015,20220623,4,NaN,용인시,백암면,93830213,127.42372,37.133545,NaN,NaN,NaN,3,NaN,FEMALE,12.0,EV라운지,5.022,right_only


In [55]:
aorb.loc[aorb['cell_id']==93782250]
# .sort_values(by='base_dt')
# .to_csv('test.csv',index=False)

,base_dt,dow,mega_nm,ccw_nm,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender_x,age_x,app_web_x,time_zone,count_cust_x,gender_y,age_y,app_web_y,count_cust_y,_merge
132454,20220603,5,경기도,용인시,백암면,93782250,127.42202,37.14887,FEMALE,12.0,EV라운지,3,6.577,NaN,NaN,NaN,NaN,left_only


In [56]:
ls = set(aorb.loc[aorb['_merge']=='left_only']['cell_id']) 
rs = set(aorb.loc[aorb['_merge']=='right_only']['cell_id'])
ls & rs
# 몇몇 cell은 하나의 데이터에만 속해있는 경우 있어 확인해봤더니 cell에 따라 나뉘는 것도 아님

{85975047,
 84951057,
 84951061,
 88023062,
 84951063,
 88023067,
 84615198,
 83927079,
 87670825,
 84951082,
 84615211,
 84951085,
 84615214,
 84615218,
 84615219,
 86999096,
 84615232,
 84615233,
 84615237,
 86310988,
 86310991,
 84615251,
 83591254,
 86310998,
 86311000,
 86311007,
 86311010,
 86311011,
 83591268,
 83927140,
 83591270,
 84951141,
 86311017,
 86311018,
 83591275,
 84951147,
 86646894,
 86646895,
 84951150,
 84951153,
 84951156,
 84951168,
 88006785,
 84951183,
 85287055,
 84951187,
 88006804,
 83927194,
 83927195,
 84951195,
 83927197,
 85287068,
 85287069,
 83927206,
 83927207,
 83927210,
 84951210,
 84951214,
 83927215,
 85622962,
 83927219,
 83591353,
 83927227,
 85287102,
 85287107,
 83927245,
 85287118,
 83927256,
 88006874,
 83927263,
 83927270,
 83927271,
 88006886,
 85287176,
 85287177,
 85287185,
 88342803,
 84263189,
 85287197,
 88342817,
 84263201,
 84263205,
 84263209,
 84263214,
 84263215,
 89030960,
 84263216,
 88342835,
 84263224,
 84263226,
 86630714,

### 도로명주소,단지 정보 결합

In [184]:
road_address = pd.read_csv("C:/Users/bhg/Desktop/2022빅콘/road_address.csv",encoding='cp949')
road_address.fillna(1,inplace=True)
# merge시 nan끼리 결합되지 않도록
merged_address = pd.merge(merged2_y,road_address,on=['cell_id','cell_xcrd','cell_ycrd'])
# merged2_y는 용인만, road_address는 전체 격자
# 각자 18000여개, 26000여개

In [185]:
merged_address.info()
# 약 21만 행 결합

<class 'pandas.core.frame.DataFrame'>
Int64Index: 630228 entries, 0 to 630227
Data columns (total 21 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   base_dt       630228 non-null  int64  
 1   dow           630228 non-null  int64  
 2   ccw_nm        630228 non-null  object 
 3   adng_nm       630228 non-null  object 
 4   cell_id       630228 non-null  int64  
 5   cell_xcrd     630228 non-null  float64
 6   cell_ycrd     630228 non-null  float64
 7   gender_x      478809 non-null  object 
 8   age_x         478809 non-null  float64
 9   app_web_x     478809 non-null  object 
 10  time_zone     630228 non-null  int64  
 11  count_cust_x  478809 non-null  float64
 12  gender_y      475877 non-null  object 
 13  age_y         475877 non-null  float64
 14  app_web_y     475877 non-null  object 
 15  count_cust_y  475877 non-null  float64
 16  gender        414271 non-null  object 
 17  age           414271 non-null  float64
 18  app_

In [88]:
# merged_address.to_csv('merged_address.csv',encoding='cp949')

In [186]:
housing = pd.read_csv("C:/Users/bhg/Desktop/2022빅콘/조사 데이터/용인시_단지기본정보_220901.csv")
housing = housing[['단지명','단지분류','법정동주소', '도로명주소','세대수','총주차대수', '지상주차대수', '지하주차대수']]
housing
# 단지 468개

,단지명,단지분류,법정동주소,도로명주소,세대수,총주차대수,지상주차대수,지하주차대수
0,역북신원아침도시아파트,아파트,경기도 용인처인구 역북동 795 역북신원아침도시아파트,경기도 용인시 처인구 낙은로 11,452,490,0,490
1,양지포레스토리,연립주택,경기도 용인처인구 양지면 남곡리 898 양지포레스토리,경기도 용인시 처인구 은이로 85,174,185,185,0
2,양우내안애에듀퍼스트,아파트,경기도 용인처인구 고림동 1014 양우내안애에듀퍼스트,경기도 용인시 처인구 경안천로 196-1,"1,098","1,217",8,1209
3,힉스유타워 아파트,아파트,경기도 용인기흥구 영덕동 1315 힉스유타워 아파트,경기도 용인시 기흥구 중부대로 184,230,265,0,265
4,성복역 KCC스위첸,아파트,경기도 용인수지구 성복동 167 성복역 KCC스위첸,경기도 용인시 수지구 성복1로 62,188,212,18,194
...,...,...,...,...,...,...,...,...
463,죽전7차현대홈타운1단지,아파트,경기도 용인수지구 죽전동 1151 죽전7차현대홈타운1단지,NaN,162,178,61,117
464,기흥더샵프라임뷰,아파트,경기도 용인기흥구 신갈동 733 기흥더샵프라임뷰,NaN,612,775,0,775
465,기흥역롯데캐슬스카이,주상복합,경기도 용인기흥구 신갈동 63 기흥역롯데캐슬스카이,NaN,625,797,0,797
466,힐스테이트 서천 아파트,아파트,경기도 용인기흥구 서천동 801 힐스테이트 서천 아파트,NaN,754,912,4,908


In [197]:
grid_housing = pd.merge(road_address,housing, how='outer',left_on = 'Address',right_on='도로명주소')
grid_housing

,cell_id,cell_xcrd,cell_ycrd,Address,단지명,단지분류,법정동주소,도로명주소,세대수,총주차대수,지상주차대수,지하주차대수
0,86135034.0,127.152250,37.286300,경기도 용인시 기흥구 동백8로 9,호수마을자연앤데시앙,아파트,경기도 용인기흥구 동백동 572 호수마을자연앤데시앙,경기도 용인시 기흥구 동백8로 9,432,486,188.0,298.0
1,86119036.0,127.151690,37.286750,경기도 용인시 기흥구 동백8로 9,호수마을자연앤데시앙,아파트,경기도 용인기흥구 동백동 572 호수마을자연앤데시앙,경기도 용인시 기흥구 동백8로 9,432,486,188.0,298.0
2,86119035.0,127.151690,37.286297,경기도 용인시 기흥구 동백8로 9,호수마을자연앤데시앙,아파트,경기도 용인기흥구 동백동 572 호수마을자연앤데시앙,경기도 용인시 기흥구 동백8로 9,432,486,188.0,298.0
3,86103037.0,127.151120,37.286747,경기도 용인시 기흥구 동백8로 9,호수마을자연앤데시앙,아파트,경기도 용인기흥구 동백동 572 호수마을자연앤데시앙,경기도 용인시 기흥구 동백8로 9,432,486,188.0,298.0
4,86103038.0,127.151115,37.287197,경기도 용인시 기흥구 동백8로 9,호수마을자연앤데시앙,아파트,경기도 용인기흥구 동백동 572 호수마을자연앤데시앙,경기도 용인시 기흥구 동백8로 9,432,486,188.0,298.0
...,...,...,...,...,...,...,...,...,...,...,...,...
26761,NaN,NaN,NaN,NaN,죽전7차현대홈타운1단지,아파트,경기도 용인수지구 죽전동 1151 죽전7차현대홈타운1단지,NaN,162,178,61.0,117.0
26762,NaN,NaN,NaN,NaN,기흥더샵프라임뷰,아파트,경기도 용인기흥구 신갈동 733 기흥더샵프라임뷰,NaN,612,775,0.0,775.0
26763,NaN,NaN,NaN,NaN,기흥역롯데캐슬스카이,주상복합,경기도 용인기흥구 신갈동 63 기흥역롯데캐슬스카이,NaN,625,797,0.0,797.0
26764,NaN,NaN,NaN,NaN,힐스테이트 서천 아파트,아파트,경기도 용인기흥구 서천동 801 힐스테이트 서천 아파트,NaN,754,912,4.0,908.0


In [199]:
grid_housing.info()
# 3786행 단지로 구분

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26766 entries, 0 to 26765
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   cell_id    26715 non-null  float64
 1   cell_xcrd  26715 non-null  float64
 2   cell_ycrd  26715 non-null  float64
 3   Address    26715 non-null  object 
 4   단지명        3786 non-null   object 
 5   단지분류       3785 non-null   object 
 6   법정동주소      3786 non-null   object 
 7   도로명주소      3768 non-null   object 
 8   세대수        3786 non-null   object 
 9   총주차대수      3786 non-null   object 
 10  지상주차대수     3786 non-null   float64
 11  지하주차대수     3786 non-null   float64
dtypes: float64(5), object(7)
memory usage: 2.7+ MB


In [200]:
grid_housing.to_csv('grid_housing.csv',encoding='cp949')

In [42]:
merged2_y

,base_dt,dow,ccw_nm,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender_x,age_x,app_web_x,...,count_cust_x,gender_y,age_y,app_web_y,count_cust_y,mega_nm,gender,age,app_web,count_cust
0,20220605,7,용인시,동백1동,86135034,127.15225,37.286300,MALE,6.0,EV라운지,...,3.522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20220625,6,용인시,상현2동,84375196,127.09008,37.309536,MALE,8.0,파워큐브차저,...,4.231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20220625,6,용인시,상현2동,84375196,127.09008,37.309536,MALE,8.0,하이차저,...,8.462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20220614,2,용인시,포곡읍,87958893,127.21660,37.274303,FEMALE,14.0,EV라운지,...,18.440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20220619,7,용인시,신봉동,84119243,127.08097,37.323480,MALE,13.0,EV라운지,...,4.335,MALE,13.0,EV라운지,4.335,경기도,MALE,13.0,EV라운지,4.335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630223,20220620,1,용인시,양지면,90182654,127.29511,37.229393,NaN,NaN,NaN,...,NaN,FEMALE,4.0,EV라운지,3.274,경기도,FEMALE,4.0,EV라운지,3.274
630224,20220608,3,용인시,남사읍,85878686,127.14400,37.122220,NaN,NaN,NaN,...,NaN,MALE,9.0,EV라운지,4.140,NaN,NaN,NaN,NaN,NaN
630225,20220628,2,용인시,성복동,83687264,127.06575,37.320717,NaN,NaN,NaN,...,NaN,FEMALE,8.0,EV라운지,5.266,NaN,NaN,NaN,NaN,NaN
630226,20220628,2,용인시,신봉동,84103247,127.08040,37.324825,NaN,NaN,NaN,...,NaN,MALE,3.0,EV라운지,21.564,NaN,NaN,NaN,NaN,NaN


In [58]:
housing_df = pd.merge(merged2_y, grid_housing.loc[grid_housing['_merge']=='both'],on=['cell_id','cell_xcrd','cell_ycrd'],how='left')

In [61]:
housing_df.isnull().sum()

base_dt               0
dow                   0
ccw_nm                0
adng_nm               0
cell_id               0
cell_xcrd             0
cell_ycrd             0
gender_x        1412530
age_x           1412530
app_web_x       1412530
time_zone             0
count_cust_x    1412530
gender_y        1182681
age_y           1182681
app_web_y       1182681
count_cust_y    1182681
mega_nm         1777458
gender          1777458
age             1777458
app_web         1777458
count_cust      1777458
Address         3514573
단지명              148879
법정동주소            148879
도로명주소           3514573
세대수              148879
총주차대수            148879
지상주차대수           148879
지하주차대수           148879
_merge           148879
dtype: int64

In [55]:
merged2_y

,base_dt,dow,ccw_nm,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender_x,age_x,app_web_x,...,count_cust_x,gender_y,age_y,app_web_y,count_cust_y,mega_nm,gender,age,app_web,count_cust
0,20220605,7,용인시,동백1동,86135034,127.15225,37.286300,MALE,6.0,EV라운지,...,3.522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20220625,6,용인시,상현2동,84375196,127.09008,37.309536,MALE,8.0,파워큐브차저,...,4.231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20220625,6,용인시,상현2동,84375196,127.09008,37.309536,MALE,8.0,하이차저,...,8.462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20220614,2,용인시,포곡읍,87958893,127.21660,37.274303,FEMALE,14.0,EV라운지,...,18.440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20220619,7,용인시,신봉동,84119243,127.08097,37.323480,MALE,13.0,EV라운지,...,4.335,MALE,13.0,EV라운지,4.335,경기도,MALE,13.0,EV라운지,4.335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630223,20220620,1,용인시,양지면,90182654,127.29511,37.229393,NaN,NaN,NaN,...,NaN,FEMALE,4.0,EV라운지,3.274,경기도,FEMALE,4.0,EV라운지,3.274
630224,20220608,3,용인시,남사읍,85878686,127.14400,37.122220,NaN,NaN,NaN,...,NaN,MALE,9.0,EV라운지,4.140,NaN,NaN,NaN,NaN,NaN
630225,20220628,2,용인시,성복동,83687264,127.06575,37.320717,NaN,NaN,NaN,...,NaN,FEMALE,8.0,EV라운지,5.266,NaN,NaN,NaN,NaN,NaN
630226,20220628,2,용인시,신봉동,84103247,127.08040,37.324825,NaN,NaN,NaN,...,NaN,MALE,3.0,EV라운지,21.564,NaN,NaN,NaN,NaN,NaN


In [52]:
housing_df

,base_dt,dow,ccw_nm,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender_x,age_x,app_web_x,...,count_cust,Address,단지명,법정동주소,도로명주소,세대수,총주차대수,지상주차대수,지하주차대수,_merge
0,20220605,7,용인시,동백1동,86135034,127.15225,37.286300,MALE,6.0,EV라운지,...,NaN,경기도 용인시 기흥구 동백8로 9,호수마을자연앤데시앙,경기도 용인기흥구 동백동 572 호수마을자연앤데시앙,경기도 용인시 기흥구 동백8로 9,432,486,188.0,298.0,both
1,20220613,1,용인시,동백1동,86135034,127.15225,37.286300,MALE,6.0,EV라운지,...,NaN,경기도 용인시 기흥구 동백8로 9,호수마을자연앤데시앙,경기도 용인기흥구 동백동 572 호수마을자연앤데시앙,경기도 용인시 기흥구 동백8로 9,432,486,188.0,298.0,both
2,20220611,6,용인시,동백1동,86135034,127.15225,37.286300,MALE,6.0,EV라운지,...,3.456,경기도 용인시 기흥구 동백8로 9,호수마을자연앤데시앙,경기도 용인기흥구 동백동 572 호수마을자연앤데시앙,경기도 용인시 기흥구 동백8로 9,432,486,188.0,298.0,both
3,20220611,6,용인시,동백1동,86135034,127.15225,37.286300,MALE,4.0,EV라운지,...,3.456,경기도 용인시 기흥구 동백8로 9,호수마을자연앤데시앙,경기도 용인기흥구 동백동 572 호수마을자연앤데시앙,경기도 용인시 기흥구 동백8로 9,432,486,188.0,298.0,both
4,20220619,7,용인시,동백1동,86135034,127.15225,37.286300,MALE,4.0,EV라운지,...,6.912,경기도 용인시 기흥구 동백8로 9,호수마을자연앤데시앙,경기도 용인기흥구 동백동 572 호수마을자연앤데시앙,경기도 용인시 기흥구 동백8로 9,432,486,188.0,298.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3660055,20220619,7,용인시,서농동,83863058,127.07246,37.232857,NaN,NaN,NaN,...,NaN,NaN,죽전7차현대홈타운1단지,경기도 용인수지구 죽전동 1151 죽전7차현대홈타운1단지,NaN,162,178,61.0,117.0,both
3660056,20220619,7,용인시,서농동,83863058,127.07246,37.232857,NaN,NaN,NaN,...,NaN,NaN,기흥더샵프라임뷰,경기도 용인기흥구 신갈동 733 기흥더샵프라임뷰,NaN,612,775,0.0,775.0,both
3660057,20220619,7,용인시,서농동,83863058,127.07246,37.232857,NaN,NaN,NaN,...,NaN,NaN,기흥역롯데캐슬스카이,경기도 용인기흥구 신갈동 63 기흥역롯데캐슬스카이,NaN,625,797,0.0,797.0,both
3660058,20220619,7,용인시,서농동,83863058,127.07246,37.232857,NaN,NaN,NaN,...,NaN,NaN,힐스테이트 서천 아파트,경기도 용인기흥구 서천동 801 힐스테이트 서천 아파트,NaN,754,912,4.0,908.0,both


In [53]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3660060 entries, 0 to 3660059
Data columns (total 30 columns):
 #   Column        Dtype   
---  ------        -----   
 0   base_dt       int64   
 1   dow           int64   
 2   ccw_nm        object  
 3   adng_nm       object  
 4   cell_id       int64   
 5   cell_xcrd     float64 
 6   cell_ycrd     float64 
 7   gender_x      object  
 8   age_x         float64 
 9   app_web_x     object  
 10  time_zone     int64   
 11  count_cust_x  float64 
 12  gender_y      object  
 13  age_y         float64 
 14  app_web_y     object  
 15  count_cust_y  float64 
 16  mega_nm       object  
 17  gender        object  
 18  age           float64 
 19  app_web       object  
 20  count_cust    float64 
 21  Address       object  
 22  단지명           object  
 23  법정동주소         object  
 24  도로명주소         object  
 25  세대수           object  
 26  총주차대수         object  
 27  지상주차대수        float64 
 28  지하주차대수        float64 
 29  _merge        

### 도로명주소, 상권 결합

In [38]:
golmok = pd.read_csv("C:/Users/bhg/Desktop/2022빅콘/조사 데이터/골목상권현황.csv",encoding='cp949')
baldal = pd.read_csv("C:/Users/bhg/Desktop/2022빅콘/조사 데이터/발달상권현황.csv",encoding='cp949')

In [39]:
grid_com = grid_housing.loc[grid_housing['_merge']=='left_only'] # 찾은 도로명 주소가 단지와 연결되지 않는게 약 9000행
grid_com = grid_com[['cell_id','cell_xcrd','cell_ycrd']]

In [40]:
grid_com.to_csv('grid_com.csv')